In [ ]:
import math

import numpy as np
import pandas as pd
import holoviews as hv

from colorcet import fire
from datashader.bundling import directly_connect_edges, hammer_bundle

from holoviews.operation.datashader import datashade, dynspread
from holoviews.operation import decimate

from dask.distributed import Client
from fa2 import ForceAtlas2
import networkx as nx
import cudf
import cugraph
from random import randint


datafile='/raid/cugraph_datasets/test/datasets/citationCiteseer.mtx'
ITERATIONS=500
THETA=1.2
OPTIMIZE=False

client = Client()
hv.notebook_extension('bokeh','matplotlib')
decimate.max_samples=20000
dynspread.threshold=0.01
datashade.cmap=fire[40:]
sz = dict(width=150,height=150)
%opts RGB [xaxis=None yaxis=None show_grid=False bgcolor="black"]

/home/hlinsenmaier/anaconda3/envs/cugraph_dev/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [ ]:
gdf = cudf.read_csv(datafile, delimiter=' ', comment='%')
gdf.columns=["source", "destination", "weights"]
gdf["source"] = gdf["source"] - 1
gdf["destination"] = gdf["destination"] - 1
#gdf

In [ ]:
G = cugraph.Graph()
G.from_cudf_edgelist(gdf)
G.view_edge_list()

In [ ]:
fa2 = cugraph.layout.force_atlas2(G, max_iter=ITERATIONS,# pos_list=cu_pos_list,
                barnes_hut_optimize=OPTIMIZE, barnes_hut_theta=THETA, verbose=False)
gdf.columns=["source", "target", "weights"]
fa2 = fa2.to_pandas()
fa2

In [ ]:
connected = directly_connect_edges(fa2, gdf.to_pandas())
connected

In [ ]:
%%opts RGB [tools=["hover"] width=800 height=800]

r_direct = hv.Curve(connected, label="Direct")
datashade(r_direct)

# Networkx

In [53]:
gdf.columns=["source", "destination", "weights"]
G_nx = nx.from_pandas_dataframe(gdf.to_pandas(), source='source', target='destination')

forceatlas2 = ForceAtlas2(
        # Behavior alternatives
        outboundAttractionDistribution=True,  # Dissuade hubs
        linLogMode=False,  # NOT IMPLEMENTED
        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
        edgeWeightInfluence=1.0,

        # Performance
        jitterTolerance=1.0,  # Tolerance
        barnesHutOptimize=OPTIMIZE,
        barnesHutTheta=THETA,
        multiThreaded=False,  # NOT IMPLEMENTED

        # Tuning
        scalingRatio=2.0,
        strongGravityMode=False,
        gravity=1.0,

        # Log
        verbose=False)

positions = forceatlas2.forceatlas2_networkx_layout(G_nx, #pos=pos_list,
        iterations=ITERATIONS)

In [54]:
npos = pd.DataFrame(positions).T
npos.columns=["x","y"]
npos

,x,y
1.0,-2539.510453,-261.062639
0.0,-2534.911722,-257.992254
1084.0,-2534.488406,-263.379819
946.0,-2544.710136,-261.479863
3.0,-2543.521178,37.644066
...,...,...
1584.0,-147.212050,-802.060124
1583.0,-134.516595,-806.812610
1586.0,-766.123934,-1278.504858
1585.0,-758.697939,-1275.232348


In [55]:
gdf.columns=["source", "target", "weights"]
connected = directly_connect_edges(npos, gdf.to_pandas())
connected

,x,y
0,-2539.510453,-261.062639
1,-2534.911722,-257.992254
2,NaN,NaN
3,-2534.488406,-263.379819
4,-2534.911722,-257.992254
...,...,...
8221,-758.697939,-1275.232348
8222,NaN,NaN
8223,-763.058701,-1276.204515
8224,-758.697939,-1275.232348


In [56]:
%%opts RGB [tools=["hover"] width=800 height=800]

r_direct = hv.Curve(connected, label="Direct")
datashade(r_direct)

:DynamicMap   []
   :RGB   [x,y]   (R,G,B,A)